# Preprocess Traning Data for GEN1 Training 
We create a new txt file "data/md/combined1/train1_combined.txt" about the same size as the original training file "data/hd/prepro/combined0/train_combined.txt" 
of which 3/4 or 75% is randomly picked from the original text file and 1/4 or 25% is randomly picked from the file containing the synthetic data from GEN0, "data/sd/gen0/gen0_sd.txt"

In [1]:
import random

def read_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read().strip().split('\n\n')
    return data

def write_data(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write('\n\n'.join(data))

# Load data
original_data = read_data('./data/hd/prepro/combined0/train_combined.txt')
synthetic_data = read_data('./data/sd/gen0/gen0_sd.txt')

# Determine portions
three_quarters_len = int(0.75 * len(original_data))
one_quarter_len = int(0.25 * len(original_data))

# Randomly sample data
selected_original = random.sample(original_data, three_quarters_len)
selected_synthetic = random.sample(synthetic_data, one_quarter_len)

# Combine and shuffle
combined_data = selected_original + selected_synthetic
random.shuffle(combined_data)

# Write combined data to a new file
write_data(combined_data, './data/md/combined1/train1_combined.txt')
print("Data has been successfully combined and written to train1_combined.txt.")


Data has been successfully combined and written to train1_combined.txt.


(Optional think it is more correct to keep the same validation set so that we can also compare the different gens performance metrics.)
We follow the same logic to create a hybrid validation dataset as well.

In [2]:
import random

# Load data
original_data = read_data('./data/hd/prepro/combined0/valid_combined.txt')
synthetic_data = read_data('./data/sd/gen0/gen0_sd.txt')

# Determine portions
three_quarters_len = int(0.75 * len(original_data))
one_quarter_len = int(0.25 * len(original_data))

# Randomly sample data
selected_original = random.sample(original_data, three_quarters_len)
selected_synthetic = random.sample(synthetic_data, one_quarter_len)

# Combine and shuffle
combined_data = selected_original + selected_synthetic
random.shuffle(combined_data)

# Write combined data to a new file
write_data(combined_data, './data/md/combined1/valid1_combined.txt')
print("Data has been successfully combined and written to train1_combined.txt.")


Data has been successfully combined and written to train1_combined.txt.


# Finetuning
### Step 2

Finetune the model GEN1 for 5 epochs with the combined txt file containing 3/4th real data and 1/4 synthetic data.

In [ ]:
!deepspeed run_clm.py \
    --model_name_or_path distilgpt2 \
    --train_file data/md/prepro/combined1/train1_combined.txt \
    --validation_file data/hd/prepro/combined0/valid_combined.txt \
    --do_train \
    --do_eval \
    --output_dir ./models/distilgpt2-finetuned_gen1 \
    --num_train_epochs 5 \
    --save_strategy epoch \
    --learning_rate 5e-5 \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --deepspeed ds_config.json \
    --resume_from_checkpoint ./models/distilgpt2-finetuned_gen0/checkpoint-27765
